In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from re import search
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [ ]:
full_list = []
info_flight = []
headers = {'user-agent': 'Mozilla/5.0'}

pages = 10

for i in range(pages):
    arrival_list = BeautifulSoup(requests.get('https://uk.flightaware.com/live/airport/LPPT/arrivals?;offset={};order=actualarrivaltime;sort=DESC'.format(i*20), 
                                              headers = headers).text, 'html.parser')
    arrival_list = arrival_list.find_all('table',{'class':'prettyTable fullWidth'})[0]
    arrival_list = arrival_list.find_all('tr')

    for i in arrival_list[2:]:
        flight_num = i.contents[0].text[1:]
        url_flight = 'https://uk.flightaware.com' + i.contents[0].a['href']
        url_tracklog = str(requests.get(url_flight, headers = headers).url)
        url_tracklog = url_tracklog + '/tracklog'
        list_flight = []
        list_flight = BeautifulSoup(requests.get(url_tracklog, headers = headers).text, 'html.parser')
        list_flight = list_flight.find_all('table',{'class':'prettyTable fullWidth'})[0]
        list_flight = list_flight.find_all('tr', {'class': re.compile("smallrow[12]")})

        url_flight = requests.get(url_tracklog, allow_redirects=True)

        pos = re.search("history/", url_flight.url).end()
        day = url_flight.url[pos:pos+8]

        if list_flight[3].contents[1].contents[0].text[:3] != list_flight[-1].contents[1].contents[0].text[:3]:
            overnight = True
        else:
            overnight = False
        for j in list_flight:
            if not 'flight' in j['class'][-1]:
                if overnight == True:
                    day = datetime.strftime(datetime.strptime(url_flight.url[pos:pos+8], "%Y%m%d") + timedelta(days=1), '%Y%m%d')
                else: 
                    day = url_flight.url[pos:pos+8]
            
                flight_date = flight_num + '_' + day
                if datetime.strptime(datetime.strftime(datetime.strptime(day, "%Y%m%d") + timedelta(days=-1), '%Y%m%d'), "%Y%m%d").strftime('%A')[:3] == j.contents[1].contents[0].text[:3]:
                    day_adjustment = datetime.strftime(datetime.strptime(day, "%Y%m%d") + timedelta(days=-1), '%Y%m%d')
                    event = day_adjustment + j.contents[1].contents[0].text[3:12]
                else:
                    event = day + j.contents[1].contents[0].text[3:12]
                latitude = j.contents[3].contents[0].text
                longitude = j.contents[5].contents[0].text
                mph = j.contents[13].text
                altitude = j.contents[15].contents[0].text
                info_flight.append((flight_date, event, latitude, longitude, mph, altitude))

        if len(i.contents[1].text) > 0:
            aircraft = i.contents[1].text
        else:
            aircraft = np.nan
        if i.contents[2].find('a') is not None:
            origin = i.contents[2].find('a').text
        else:
            origin = np.nan
        arrival = day + i.contents[4].text[3:9]
        id_flight = flight_date
        full_list.append((flight_num, aircraft, origin, arrival, id_flight)) 

df_flights = pd.DataFrame(full_list, columns=['flight', 'aircraft', 'origin', 'arrival', 'id_flight'])
df_flights.to_csv('flights.csv')
df_info_flight = pd.DataFrame(info_flight, columns = ['flight_date', 'event', 'latitude', 'longitude', 'mph', 'altitude'])
df_info_flight.to_csv('info_flight.csv')